In [ ]:
import numpy as np
from torch.utils.data import Dataset, DataLoader
import re
import torch
import torchvision
import math
#Information
# epoch = one forward and backward pass of ALL training samples
# batch_size = number of training samples used in one forward/backward pass
# number of iterations = number of passes, each pass (forward+backward) using [batch_size] number of sampes
# e.g : 100 samples, batch_size=20 -> 100/20=5 iterations for 1 epoch

### **DATA**

In [ ]:
def lirefile(file_name):
    data=[]
    target=[]
    
    
    with open(file_name, 'r', encoding='utf-8') as raw_file:
        for i,line in enumerate(raw_file):
            line_content = re.sub('[\n\t]', '', line)
            if line_content not in ["end",'<>']:
                d=line_content.split(" ")
                data[-1].append(d[0])
                target[-1].append(d[1])
            elif line_content in ['<>']:
                data.append([])
                target.append([])
    return data , target

In [ ]:
#one hot
ac_am_dict = {
    "1" :[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "A": [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "C": [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "D": [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "E": [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "F": [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "G": [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "H": [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "I": [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "K": [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "L": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "M": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "N": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "P": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    "Q": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    "R": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    "S": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    "T": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    "V": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    "W": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    "Y": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    
}

targ_dict = {
    
    "_": [1,0,0],
    "e": [0,1,0],
    "h": [0,0,1],
    
}

In [ ]:
def transformAcideAminToSerieFive(data):
      cpt=0
      array=[]
      Finale = []
      for row in data:
          
        array=[]
        id=0
        cpt=cpt+(len(row))
        
        for id, val in enumerate(row):
                  if id==0   : 
                    array.append(ac_am_dict["1"])
                    array.append(ac_am_dict["1"]) 
                    array.append(ac_am_dict[val])
                    
                  elif id==len(row)-1 :
                    array.append(ac_am_dict[val])
                    array.append(ac_am_dict["1"])
                    array.append(ac_am_dict["1"]) 
                    
                  
                  else :
                  
                      array.append(ac_am_dict[val])
      
        for idx, val in enumerate(array):
              el=[]
              if (idx+1 < len(array) and idx - 1 >= 0 and idx+2 < len(array) and idx - 2 >= 0 ):
                  avant2=idx - 2
                  avant1=idx - 1
                  cur=idx
                  apres1=idx+1
                  apres2=idx+2
                  
                  #el = np.concatenate((array[avant2],array[avant1],array[cur],array[apres2],array[apres1]))
                  el = [*array[avant2],*array[avant1],*array[cur],*array[apres2],*array[apres1]]
                  #print(el)
                  Finale.append(el)
      print('nombre d\'acide aminé',cpt)
      return Finale   

In [ ]:
def TransformTarget(label):
    array2=[]
    FinaleTarget = []
    for row in label:
        
        array2=[]

        
        for va in row:

              FinaleTarget.append(targ_dict[va])
     
    return FinaleTarget

In [ ]:
class Dataset(Dataset):

    def __init__(self,FinaleX,FinaleY):
        # Initialize data, download, etc.
        # read with numpy or pandas
        

        # here the first column is the class label, the rest are the features
        lst1 = [torch.from_numpy(np.array(item)).type(torch.float) for item in FinaleX]
        print(type(lst1))
        self.x_data = lst1
        lst2=[]
        for item in FinaleY :
            lst2.append(torch.from_numpy(np.array(item)).type(torch.float))
        
        self.y_data = lst2

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return len(self.y_data)





**(Train et validation)**

In [ ]:
dataTrain , labelTrain =lirefile("/content/drive/MyDrive/AvignonM1_ApplicationIA/DATA/protein-secondary-structure (1).train")
print(len(dataTrain),len(labelTrain))


TrainFinaleX=transformAcideAminToSerieFive(dataTrain)    
#Print     
print(len(TrainFinaleX))
print(np.array(TrainFinaleX[0:10]))


TrainFinaleY=TransformTarget(labelTrain)
#print    
print('nombre de labels',len(TrainFinaleY))
print(len(TrainFinaleY))
#TrainFinaleY[0:150]




111 111
nombre d'acide aminé 18105
18105
[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]]
nombre de labels 18105
18105


In [ ]:
#Create Dataset and call Dataloader
dataset = Dataset(TrainFinaleX,TrainFinaleY)
first_data = dataset[1]
features, labels = first_data
print(first_data)



#split datatrain and validation
train, valid = torch.utils.data.random_split(dataset,[14505,3600])#réduction de 20% les données d'entrainement
print(valid)




<class 'list'>
(tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]), tensor([1., 0., 0.]))


In [ ]:
train_loader = DataLoader(dataset=train,batch_size=64, shuffle=False)
val_loader=DataLoader(dataset=valid,batch_size=64, shuffle=False)

**(Test)**

In [ ]:
dataTest , labelTest =lirefile("/content/drive/MyDrive/AvignonM1_ApplicationIA/DATA/protein-secondary-structure.test")
print(len(dataTest),len(labelTest))

TestFinaleX=transformAcideAminToSerieFive(dataTest)    
#Print     
print(len(TestFinaleX))
#np.array(TrainFinaleX[0:10])


TestFinaleY=TransformTarget(labelTest)
#print    
print('nombre de labels',len(TestFinaleY))
print(len(TestFinaleY))
#TestFinaleY[0:150]




17 17
nombre d'acide aminé 3520
3520
nombre de labels 3520
3520


In [ ]:
#Create Dataset and call Dataloader

dataset = Dataset(TestFinaleX,TestFinaleY)
test_loader = DataLoader(dataset=dataset,batch_size=64, shuffle=False)

<class 'list'>


# **Model**

In [ ]:
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

Model 1

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class NeuralNetwork(nn.Module):
    def __init__(self):

        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(22*5, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 3),
            
        )
        self.drops = nn.Dropout(0.0)

    
        

    def forward(self, x):
        x = self.flatten(x)
        x=self.drops(x)
        logits = self.linear_relu_stack(x)
        out = F.softmax(logits)
        '''print(type(out))
        print(out.shape)'''
        return out

model = NeuralNetwork()
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=110, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=3, bias=True)
  )
  (drops): Dropout(p=0.0, inplace=False)
)


**Train and test loop**

In [ ]:
learning_rate = 0.1

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        #y=torch.flatten(y)print(y)print(pred)
        #torch.from_numpy
  
        pred = model(X)
        loss = loss_fn(pred, y)
        print(loss)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    
        

Partie Validation

In [ ]:
def Val_loop(dataloader, model, loss_fn):
    valid_loss,correct = 0,0
    min_valid_loss = np.inf
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()     # Optional when not using Model Specific layer
    
    for X, y in dataloader:
        # Transfer Data to GPU if available
        
         
        # Forward Pass
        pred = model(X)
        # Find the Loss
        loss = loss_fn(pred,y)
        # Calculate Loss
        valid_loss += loss.item()
        correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
    loss /= num_batches
    correct /= size
    if min_valid_loss > valid_loss:
        print("loss loss loss",min_valid_loss,valid_loss)
        
        min_valid_loss = valid_loss
         
        # Saving State Dict
        torch.save(model, '/content/drive/MyDrive/AvignonM1_ApplicationIA/model.pth')
    
    correct /= size
    print(f"Test Error: \n validation Accuracy: {(100*correct):>0.1f}%, Avg loss: {loss:>8f} \n")

In [ ]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            #y=torch.flatten(y)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            print("testLoss",test_loss)
            correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate,momentum=0.9)
#optimizer=torch.optim.Adam(model.parameters(),lr=0.005,weight_decay=0)  # 54% accuracy

for t in range(10):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, model, loss_fn, optimizer)
    Val_loop(val_loader, model, loss_fn)
model_=torch.load('/content/drive/MyDrive/AvignonM1_ApplicationIA/model.pth')
test_loop(test_loader, model_, loss_fn)
print("Done!")

Epoch 1
-------------------------------
tensor(1.1062, grad_fn=<DivBackward1>)
loss: 1.106191  [    0/14505]
tensor(1.1044, grad_fn=<DivBackward1>)
tensor(1.1055, grad_fn=<DivBackward1>)
tensor(1.0975, grad_fn=<DivBackward1>)
tensor(1.0958, grad_fn=<DivBackward1>)
tensor(1.0852, grad_fn=<DivBackward1>)
tensor(1.0964, grad_fn=<DivBackward1>)
tensor(1.0858, grad_fn=<DivBackward1>)
tensor(1.0627, grad_fn=<DivBackward1>)
tensor(1.0783, grad_fn=<DivBackward1>)
tensor(1.0726, grad_fn=<DivBackward1>)
tensor(1.0733, grad_fn=<DivBackward1>)
tensor(1.0498, grad_fn=<DivBackward1>)
tensor(1.0233, grad_fn=<DivBackward1>)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor(1.0481, grad_fn=<DivBackward1>)
tensor(1.0559, grad_fn=<DivBackward1>)
tensor(1.0418, grad_fn=<DivBackward1>)
tensor(1.0259, grad_fn=<DivBackward1>)
tensor(1.0120, grad_fn=<DivBackward1>)
tensor(1.0338, grad_fn=<DivBackward1>)
tensor(1.0229, grad_fn=<DivBackward1>)
tensor(1.0001, grad_fn=<DivBackward1>)
tensor(1.0472, grad_fn=<DivBackward1>)
tensor(1.0093, grad_fn=<DivBackward1>)
tensor(0.9385, grad_fn=<DivBackward1>)
tensor(1.0188, grad_fn=<DivBackward1>)
tensor(0.9687, grad_fn=<DivBackward1>)
tensor(1.0861, grad_fn=<DivBackward1>)
tensor(1.0406, grad_fn=<DivBackward1>)
tensor(1.0154, grad_fn=<DivBackward1>)
tensor(0.9272, grad_fn=<DivBackward1>)
tensor(1.0013, grad_fn=<DivBackward1>)
tensor(1.0408, grad_fn=<DivBackward1>)
tensor(1.0013, grad_fn=<DivBackward1>)
tensor(1.1058, grad_fn=<DivBackward1>)
tensor(1.0288, grad_fn=<DivBackward1>)
tensor(1.0690, grad_fn=<DivBackward1>)
tensor(0.9607, grad_fn=<DivBackward1>)
tensor(1.1461, grad_fn=<DivBackward1>)
tensor(0.9741, grad_fn=<D